<a href="https://colab.research.google.com/github/dianagromakovskaya/data-engineering-zoomcamp/blob/dlt_workshop/homework/workshop1_data_ingestion/Homework_data_talks_club_data_extraction_and_ingestion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [9]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution: First make sure that the following modules are installed:

In [4]:
#Install the dependencies
%%capture
!pip install dlt[duckdb]

In [25]:
# to do: homework :)

def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

limit = 5
generator = square_root_generator(limit)

# Question 1: What is the sum of the outputs of the generator for limit = 5?
print(sum([i for i in generator]))

8.382332347441762


In [26]:
from itertools import islice

# Question 2: What is the 13th number yielded
limit = 13
generator = square_root_generator(limit)
print(next(islice(generator, 12, None)))

3.605551275463989


In [29]:
import duckdb
import dlt

# Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')
info = pipeline.run(people_1,
										table_name="people_generator",
										write_disposition="replace")
print(info)

conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
age_sum = conn.sql("SELECT SUM(age) FROM people_generator").df()
display(age_sum)

# Append the second generator to the same table as the first.
info = pipeline.run(people_2,
										table_name="people_generator",
										write_disposition="append")
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.18 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708287599.5226922 is LOADED and contains no failed jobs


,sum(age)
0,140.0


Pipeline dlt_colab_kernel_launcher load step completed in 0.17 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708287600.1344535 is LOADED and contains no failed jobs


In [30]:
print('Loaded tables: ')
display(conn.sql("show tables"))


people = conn.sql("SELECT * FROM people_generator").df()
display(people)

# After correctly appending the data, calculate the sum of all ages of people.
age_sum = conn.sql("SELECT SUM(age) FROM people_generator").df()
display(age_sum)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_1_generator  │
│ people_generator    │
└─────────────────────┘

,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708287599.5226922,wZfaTp7EeV9j2w,None
1,2,Person_2,27,City_A,1708287599.5226922,M1sI+LxFesPiJA,None
2,3,Person_3,28,City_A,1708287599.5226922,t7Y8Y2L2uiU05A,None
3,4,Person_4,29,City_A,1708287599.5226922,soouZBsV2N6kvw,None
4,5,Person_5,30,City_A,1708287599.5226922,eHOaO3S9L34AqQ,None
5,3,Person_3,33,City_B,1708287600.1344535,A/vrRXX1KahkPA,Job_3
6,4,Person_4,34,City_B,1708287600.1344535,b697MSNGzFLW4Q,Job_4
7,5,Person_5,35,City_B,1708287600.1344535,7uYBfNSgKwJ67w,Job_5
8,6,Person_6,36,City_B,1708287600.1344535,rlFE2mVkEGRR2g,Job_6
9,7,Person_7,37,City_B,1708287600.1344535,2iv9kSA/z7oW7A,Job_7


,sum(age)
0,353.0


In [31]:
# Load your first generator first, and then load the second one with merge.
# Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.
# After loading, you should have a total of 8 records, and ID 3 should have age 33.
pipeline = dlt.pipeline(destination='duckdb', dataset_name='people')
info = pipeline.run(people_1,
										table_name="people_generator",
										write_disposition="replace",
                    primary_key="id")
print(info)

info = pipeline.run(people_2,
										table_name="people_generator",
										write_disposition="merge",
                    primary_key="id")
print(info)

people = conn.sql("SELECT * FROM people_generator").df()
display(people)

Pipeline dlt_colab_kernel_launcher load step completed in 0.18 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708287729.6228786 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.34 seconds
1 load package(s) were loaded to destination duckdb and into dataset people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708287730.2326117 is LOADED and contains no failed jobs


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708287729.6228786,YA96hia8heBGWw,None
1,2,Person_2,27,City_A,1708287729.6228786,f0+lm3I8hv08IQ,None
2,5,Person_5,35,City_B,1708287730.2326117,bPZayMDG75mytw,Job_5
3,7,Person_7,37,City_B,1708287730.2326117,6kKL6Ghye7rksA,Job_7
4,4,Person_4,34,City_B,1708287730.2326117,jUSO2qJJtkZ3Sw,Job_4
5,3,Person_3,33,City_B,1708287730.2326117,fXRd9WrIE/grrg,Job_3
6,6,Person_6,36,City_B,1708287730.2326117,t3pJVHDwGBisAw,Job_6
7,8,Person_8,38,City_B,1708287730.2326117,a1g8MmE4GVDztA,Job_8


In [32]:
# Calculate the sum of ages of all the people loaded as described above.
age_sum = conn.sql("SELECT SUM(age) FROM people_generator").df()
display(age_sum)

,sum(age)
0,266.0


Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX